In [1]:
# Import dependencies:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


#from imblearn.ensemble import BalancedRandomForestClassifier
#from sklearn.metrics import balanced_accuracy_score
#from imblearn.metrics import classification_report_imbalanced
#from imblearn.ensemble import EasyEnsembleClassifier

In [2]:
# Load the data:
file_path = Path("../resources/data/cleaned-input-data/CLEAN_KEPLER_DATASET.csv")
exoplanet_df = pd.read_csv(file_path)

print(exoplanet_df.shape)
exoplanet_df.head()

(9564, 141)


rowid     kepid kepoi_name   kepler_name koi_disposition koi_vet_stat  \
0      1  10797460  K00752.01  Kepler-227 b       CONFIRMED         Done   
1      2  10797460  K00752.02  Kepler-227 c       CONFIRMED         Done   
2      3  10811496  K00753.01           NaN       CANDIDATE         Done   
3      4  10848459  K00754.01           NaN  FALSE POSITIVE         Done   
4      5  10854555  K00755.01  Kepler-664 b       CONFIRMED         Done   

  koi_vet_date koi_pdisposition  koi_score  koi_fpflag_nt  ...  \
0    8/16/2018        CANDIDATE      1.000              0  ...   
1    8/16/2018        CANDIDATE      0.969              0  ...   
2    8/16/2018        CANDIDATE      0.000              0  ...   
3    8/16/2018   FALSE POSITIVE      0.000              0  ...   
4    8/16/2018        CANDIDATE      1.000              0  ...   

   koi_dicco_mdec  koi_dicco_mdec_err  koi_dicco_msky koi_dicco_msky_err  \
0           0.200               0.160           0.200              0.170   
1           0.000               0.480           0.390              0.360   
2          -0.034               0.070           0.042              0.072   
3           0.147               0.078           0.289              0.079   
4          -0.090               0.180           0.100              0.140   

  koi_dikco_mra  koi_dikco_mra_err  koi_dikco_mdec  koi_dikco_mdec_err  \
0         0.080              0.130           0.310               0.170   
1         0.490              0.340           0.120               0.730   
2         0.002              0.071          -0.027               0.074   
3        -0.257              0.072           0.099               0.077   
4         0.070              0.180           0.020               0.160   

   koi_dikco_msky  koi_dikco_msky_err  
0           0.320               0.160  
1           0.500               0.450  
2           0.027               0.074  
3           0.276               0.076  
4           0.070               0.200  

[5 rows x 141 columns]

In [3]:
# Count distinct values in "koi_disposition" column:
exoplanet_df['koi_disposition'].value_counts()

FALSE POSITIVE    4839
CONFIRMED         2669
CANDIDATE         2056
Name: koi_disposition, dtype: int64

In [4]:
# Create a new dataframe by selecting desired columns also set the "rowid" column as index:
new_exoplanet_df = exoplanet_df[["rowid", "kepoi_name", "koi_count", "koi_prad",
                                 "koi_teq", "koi_period", "koi_srad", 
                                 "koi_steff", "koi_smass", "koi_slogg", "koi_ror", "koi_disposition"]].set_index('rowid')


print(new_exoplanet_df.shape)
new_exoplanet_df.index.name = None
new_exoplanet_df.head()

(9564, 11)


kepoi_name  koi_count  koi_prad  koi_teq  koi_period  koi_srad  koi_steff  \
1  K00752.01          2      2.26    793.0    9.488036     0.927     5455.0   
2  K00752.02          2      2.83    443.0   54.418383     0.927     5455.0   
3  K00753.01          1     14.60    638.0   19.899140     0.868     5853.0   
4  K00754.01          1     33.46   1395.0    1.736952     0.791     5805.0   
5  K00755.01          1      2.75   1406.0    2.525592     1.046     6031.0   

   koi_smass  koi_slogg   koi_ror koi_disposition  
1      0.919      4.467  0.022344       CONFIRMED  
2      0.919      4.467  0.027954       CONFIRMED  
3      0.961      4.544  0.154046       CANDIDATE  
4      0.836      4.564  0.387394  FALSE POSITIVE  
5      1.095      4.438  0.024064       CONFIRMED

In [5]:
# Dropna from new dataframe:
new_exoplanet_df= new_exoplanet_df.dropna()
print(new_exoplanet_df.shape)
new_exoplanet_df.head()

(9201, 11)


kepoi_name  koi_count  koi_prad  koi_teq  koi_period  koi_srad  koi_steff  \
1  K00752.01          2      2.26    793.0    9.488036     0.927     5455.0   
2  K00752.02          2      2.83    443.0   54.418383     0.927     5455.0   
3  K00753.01          1     14.60    638.0   19.899140     0.868     5853.0   
4  K00754.01          1     33.46   1395.0    1.736952     0.791     5805.0   
5  K00755.01          1      2.75   1406.0    2.525592     1.046     6031.0   

   koi_smass  koi_slogg   koi_ror koi_disposition  
1      0.919      4.467  0.022344       CONFIRMED  
2      0.919      4.467  0.027954       CONFIRMED  
3      0.961      4.544  0.154046       CANDIDATE  
4      0.836      4.564  0.387394  FALSE POSITIVE  
5      1.095      4.438  0.024064       CONFIRMED

In [6]:
# Drop duplicates:
clean_exoplanet_df = new_exoplanet_df.drop_duplicates()
print(clean_exoplanet_df.shape)
clean_exoplanet_df.head()

(9201, 11)


kepoi_name  koi_count  koi_prad  koi_teq  koi_period  koi_srad  koi_steff  \
1  K00752.01          2      2.26    793.0    9.488036     0.927     5455.0   
2  K00752.02          2      2.83    443.0   54.418383     0.927     5455.0   
3  K00753.01          1     14.60    638.0   19.899140     0.868     5853.0   
4  K00754.01          1     33.46   1395.0    1.736952     0.791     5805.0   
5  K00755.01          1      2.75   1406.0    2.525592     1.046     6031.0   

   koi_smass  koi_slogg   koi_ror koi_disposition  
1      0.919      4.467  0.022344       CONFIRMED  
2      0.919      4.467  0.027954       CONFIRMED  
3      0.961      4.544  0.154046       CANDIDATE  
4      0.836      4.564  0.387394  FALSE POSITIVE  
5      1.095      4.438  0.024064       CONFIRMED

In [7]:
# Count distinct values in "koi_disposition" column:
clean_exoplanet_df['koi_disposition'].value_counts()

FALSE POSITIVE    4582
CONFIRMED         2668
CANDIDATE         1951
Name: koi_disposition, dtype: int64

In [8]:
# Give columns proper names:

#clean_exoplanet_df.columns = ['Kepler(OI) Name', 'Number of Planets', 'Planet Radius (Earth)',
#                              'Equalibrium Temp (K)', 'Orbit Period (Days)','Star Radius (Sun)',
#                              'Star Temp (K)', 'Star Mass (Sun)', 'Star Surface Gravity', 'Planet-Star Radius Ratio', 'Status']

# Change column names for ETL:

clean_exoplanet_df.columns = ['Kepler_OI_Name', 'Number_of_Planets', 'Planet_Radius_Earth',
                              'Equalibrium_Temp_K', 'Orbit_Period_Days','Star_Radius_Sun',
                              'Star_Temp_K', 'Star_Mass_Sun', 'Star_Surface_Gravity', 'Planet_Star_Radius_Ratio', 'Status']

# Stellar Surface Gravity (log10(cm s-2)
# The base-10 logarithm of the acceleration due to gravity at the surface of the star.

clean_exoplanet_df.head()

Kepler_OI_Name  Number_of_Planets  Planet_Radius_Earth  Equalibrium_Temp_K  \
1      K00752.01                  2                 2.26               793.0   
2      K00752.02                  2                 2.83               443.0   
3      K00753.01                  1                14.60               638.0   
4      K00754.01                  1                33.46              1395.0   
5      K00755.01                  1                 2.75              1406.0   

   Orbit_Period_Days  Star_Radius_Sun  Star_Temp_K  Star_Mass_Sun  \
1           9.488036            0.927       5455.0          0.919   
2          54.418383            0.927       5455.0          0.919   
3          19.899140            0.868       5853.0          0.961   
4           1.736952            0.791       5805.0          0.836   
5           2.525592            1.046       6031.0          1.095   

   Star_Surface_Gravity  Planet_Star_Radius_Ratio          Status  
1                 4.467                  0.022344       CONFIRMED  
2                 4.467                  0.027954       CONFIRMED  
3                 4.544                  0.154046       CANDIDATE  
4                 4.564                  0.387394  FALSE POSITIVE  
5                 4.438                  0.024064       CONFIRMED

In [9]:
# Replace values in 'Status' with 0 , 1 , or 2:
nm_value_df = clean_exoplanet_df.replace({'CONFIRMED':1, 'FALSE POSITIVE':0, 'CANDIDATE':2})
print(nm_value_df.shape)
nm_value_df.head()

(9201, 11)


Kepler_OI_Name  Number_of_Planets  Planet_Radius_Earth  Equalibrium_Temp_K  \
1      K00752.01                  2                 2.26               793.0   
2      K00752.02                  2                 2.83               443.0   
3      K00753.01                  1                14.60               638.0   
4      K00754.01                  1                33.46              1395.0   
5      K00755.01                  1                 2.75              1406.0   

   Orbit_Period_Days  Star_Radius_Sun  Star_Temp_K  Star_Mass_Sun  \
1           9.488036            0.927       5455.0          0.919   
2          54.418383            0.927       5455.0          0.919   
3          19.899140            0.868       5853.0          0.961   
4           1.736952            0.791       5805.0          0.836   
5           2.525592            1.046       6031.0          1.095   

   Star_Surface_Gravity  Planet_Star_Radius_Ratio  Status  
1                 4.467                  0.022344       1  
2                 4.467                  0.027954       1  
3                 4.544                  0.154046       2  
4                 4.564                  0.387394       0  
5                 4.438                  0.024064       1

In [10]:
# Check values in "Status" column:
nm_value_df['Status'].value_counts()

0    4582
1    2668
2    1951
Name: Status, dtype: int64

In [11]:
# Check data types in columns:
nm_value_df.dtypes

Kepler_OI_Name               object
Number_of_Planets             int64
Planet_Radius_Earth         float64
Equalibrium_Temp_K          float64
Orbit_Period_Days           float64
Star_Radius_Sun             float64
Star_Temp_K                 float64
Star_Mass_Sun               float64
Star_Surface_Gravity        float64
Planet_Star_Radius_Ratio    float64
Status                        int64
dtype: object

In [12]:
# Create a new DataFrame that holds only the Kepler Object of Intrest (OI) names:
names_df = nm_value_df[["Kepler_OI_Name"]]
print(names_df.shape)
names_df.head(10)

(9201, 1)


Kepler_OI_Name
1       K00752.01
2       K00752.02
3       K00753.01
4       K00754.01
5       K00755.01
6       K00756.01
7       K00756.02
8       K00756.03
9       K00114.01
10      K00757.01

In [13]:
# Drop the 'Kepler Name (OI)' column since it's not going to be used for model:
nm_value_df = nm_value_df.drop(columns=["Kepler_OI_Name"])
print(nm_value_df.shape)
nm_value_df.head(10)

(9201, 10)


Number_of_Planets  Planet_Radius_Earth  Equalibrium_Temp_K  \
1                   2                 2.26               793.0   
2                   2                 2.83               443.0   
3                   1                14.60               638.0   
4                   1                33.46              1395.0   
5                   1                 2.75              1406.0   
6                   3                 3.90               835.0   
7                   3                 2.77              1160.0   
8                   3                 1.59              1360.0   
9                   1                39.21              1342.0   
10                  3                 5.76               600.0   

    Orbit_Period_Days  Star_Radius_Sun  Star_Temp_K  Star_Mass_Sun  \
1            9.488036            0.927       5455.0          0.919   
2           54.418383            0.927       5455.0          0.919   
3           19.899140            0.868       5853.0          0.961   
4            1.736952            0.791       5805.0          0.836   
5            2.525592            1.046       6031.0          1.095   
6           11.094321            0.972       6046.0          1.053   
7            4.134435            0.972       6046.0          1.053   
8            2.566589            0.972       6046.0          1.053   
9            7.361790            1.958       6227.0          1.358   
10          16.068647            0.848       5031.0          0.801   

    Star_Surface_Gravity  Planet_Star_Radius_Ratio  Status  
1                  4.467                  0.022344       1  
2                  4.467                  0.027954       1  
3                  4.544                  0.154046       2  
4                  4.564                  0.387394       0  
5                  4.438                  0.024064       1  
6                  4.486                  0.036779       1  
7                  4.486                  0.026133       1  
8                  4.486                  0.014983       1  
9                  3.986                  0.183387       0  
10                 4.485                  0.062161       1

# Separate Candidate planets from Confirmed and False Positives:

## Create a DF for Verified Planets

In [14]:
# Create a DF for Verified Planets
verified_planets_df = nm_value_df.drop(nm_value_df.index[nm_value_df['Status'] == 2])
print(verified_planets_df.shape)
verified_planets_df.head()

(7250, 10)


Number_of_Planets  Planet_Radius_Earth  Equalibrium_Temp_K  \
1                  2                 2.26               793.0   
2                  2                 2.83               443.0   
4                  1                33.46              1395.0   
5                  1                 2.75              1406.0   
6                  3                 3.90               835.0   

   Orbit_Period_Days  Star_Radius_Sun  Star_Temp_K  Star_Mass_Sun  \
1           9.488036            0.927       5455.0          0.919   
2          54.418383            0.927       5455.0          0.919   
4           1.736952            0.791       5805.0          0.836   
5           2.525592            1.046       6031.0          1.095   
6          11.094321            0.972       6046.0          1.053   

   Star_Surface_Gravity  Planet_Star_Radius_Ratio  Status  
1                 4.467                  0.022344       1  
2                 4.467                  0.027954       1  
4                 4.564                  0.387394       0  
5                 4.438                  0.024064       1  
6                 4.486                  0.036779       1

In [15]:
# Check values in "Status" column:
verified_planets_df['Status'].value_counts()

0    4582
1    2668
Name: Status, dtype: int64

In [16]:
# Export the Dataframe as a new CSV file without the index.
verified_planets_df.to_csv("../resources/data/model-output-data/verified_planets_df.csv", index=False)
verified_planets_df.dtypes

Number_of_Planets             int64
Planet_Radius_Earth         float64
Equalibrium_Temp_K          float64
Orbit_Period_Days           float64
Star_Radius_Sun             float64
Star_Temp_K                 float64
Star_Mass_Sun               float64
Star_Surface_Gravity        float64
Planet_Star_Radius_Ratio    float64
Status                        int64
dtype: object

## Create a DF for Unverified Planets

In [17]:
# Create a DF for Unverified Planets
unverified_planets_df = nm_value_df.drop(nm_value_df.index[nm_value_df['Status'].isin([1, 0])])
print(unverified_planets_df.shape)
unverified_planets_df.head()

(1951, 10)


Number_of_Planets  Planet_Radius_Earth  Equalibrium_Temp_K  \
3                   1                14.60               638.0   
38                  1                12.21              1103.0   
59                  1                 7.51               467.0   
63                  2                19.45               734.0   
64                  3                 0.55              1272.0   

    Orbit_Period_Days  Star_Radius_Sun  Star_Temp_K  Star_Mass_Sun  \
3           19.899140            0.868       5853.0          0.961   
38           4.959319            1.082       5712.0          0.976   
59          40.419504            0.781       5446.0          0.714   
63           7.240661            0.765       5005.0          0.850   
64           3.435916            1.087       5779.0          0.941   

    Star_Surface_Gravity  Planet_Star_Radius_Ratio  Status  
3                  4.544                  0.154046       2  
38                 4.359                  0.103379       2  
59                 4.507                  0.088069       2  
63                 4.595                  0.232818       2  
64                 4.339                  0.004612       2

In [18]:
# Check values in "Status" column:
unverified_planets_df['Status'].value_counts()

2    1951
Name: Status, dtype: int64

In [19]:
# Export the Dataframe as a new CSV file without the index.
unverified_planets_df.to_csv("../resources/data/model-output-data/unverified_planets_df.csv", index=False)
unverified_planets_df.dtypes

Number_of_Planets             int64
Planet_Radius_Earth         float64
Equalibrium_Temp_K          float64
Orbit_Period_Days           float64
Star_Radius_Sun             float64
Star_Temp_K                 float64
Star_Mass_Sun               float64
Star_Surface_Gravity        float64
Planet_Star_Radius_Ratio    float64
Status                        int64
dtype: object

# Import data from AWS Deployed PostgreSQL Database

# Create a trained model to use for final prediction: 
## Split the Data into Training and Testing

### Create a Supervised Machine Learning model, use Logistic Regression // classification 1 or 0.

In [20]:
# Create our features:
X = verified_planets_df.drop("Status", axis=1)

# Create our target:
y = verified_planets_df["Status"]

In [21]:
# Split into Train and Test Sets:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify = y,  test_size= 0.33)

In [22]:
# Train the model:
lm = LogisticRegression(solver='lbfgs',
                                max_iter=300,
                                random_state=1)
lm.fit(X_train, y_train)

LogisticRegression(max_iter=300, random_state=1)

In [23]:
# Validate the model:
from sklearn.metrics import classification_report

predict = lm.predict(X_test)

# Print all results:
with np.printoptions(threshold=np.inf):
    print(predict)

[0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 1 1 0 1
 0 0 1 1 1 0 0 1 0 1 0 1 0 1 0 0 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 0 0 1 0 1 0
 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 1 0 0 0 1 0 1 1 1 0 1 0 1 0
 0 0 1 0 1 0 0 0 0 1 0 1 1 0 1 1 1 1 0 0 1 0 0 0 0 0 1 1 1 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 1
 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 1 1 0 1 1 0 1 0 1 1
 0 0 1 1 1 0 1 0 0 0 1 1 1 0 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0
 1 0 0 0 0 1 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0
 0 0 0 1 1 1 0 1 1 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 0 0 1 0
 1 1 0 0 1 0 0 0 1 0 1 1 0 1 0 1 0 1 0 1 0 1 1 1 0 0 1 1 1 0 0 0 1 1 1 0 0
 0 0 0 0 0 0 1 1 0 0 1 0 1 1 0 0 1 1 1 1 0 1 0 1 1 1 0 0 0 0 0 0 0 1 0 1 1
 1 0 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0
 0 1 0 0 0 0 0 1 0 1 0 0 1 1 0 0 1 0 1 1 0 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0
 1 0 0 1 0 1 0 1 0 0 0 0 

In [24]:
# DF for output:
t1 = pd.DataFrame({"Prediction": predict, "Actual": y_test})
t1.head(10)

Prediction  Actual
4504           0       0
1691           1       1
1818           1       1
7533           0       0
2554           0       0
5012           0       0
4451           0       0
1134           0       0
1951           1       1
2632           1       0

In [25]:
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

           0       0.90      0.86      0.88      1512
           1       0.77      0.83      0.80       881

    accuracy                           0.85      2393
   macro avg       0.83      0.84      0.84      2393
weighted avg       0.85      0.85      0.85      2393



# Predict results for unverified_planets_df:

In [26]:
# Display the 'unverified_planets_df':
unverified_planets_df.head()

Number_of_Planets  Planet_Radius_Earth  Equalibrium_Temp_K  \
3                   1                14.60               638.0   
38                  1                12.21              1103.0   
59                  1                 7.51               467.0   
63                  2                19.45               734.0   
64                  3                 0.55              1272.0   

    Orbit_Period_Days  Star_Radius_Sun  Star_Temp_K  Star_Mass_Sun  \
3           19.899140            0.868       5853.0          0.961   
38           4.959319            1.082       5712.0          0.976   
59          40.419504            0.781       5446.0          0.714   
63           7.240661            0.765       5005.0          0.850   
64           3.435916            1.087       5779.0          0.941   

    Star_Surface_Gravity  Planet_Star_Radius_Ratio  Status  
3                  4.544                  0.154046       2  
38                 4.359                  0.103379       2  
59                 4.507                  0.088069       2  
63                 4.595                  0.232818       2  
64                 4.339                  0.004612       2

In [27]:
# Drop the 'Status' column:
X_new = unverified_planets_df.drop("Status", axis=1)
X_new.head()

Number_of_Planets  Planet_Radius_Earth  Equalibrium_Temp_K  \
3                   1                14.60               638.0   
38                  1                12.21              1103.0   
59                  1                 7.51               467.0   
63                  2                19.45               734.0   
64                  3                 0.55              1272.0   

    Orbit_Period_Days  Star_Radius_Sun  Star_Temp_K  Star_Mass_Sun  \
3           19.899140            0.868       5853.0          0.961   
38           4.959319            1.082       5712.0          0.976   
59          40.419504            0.781       5446.0          0.714   
63           7.240661            0.765       5005.0          0.850   
64           3.435916            1.087       5779.0          0.941   

    Star_Surface_Gravity  Planet_Star_Radius_Ratio  
3                  4.544                  0.154046  
38                 4.359                  0.103379  
59                 4.507                  0.088069  
63                 4.595                  0.232818  
64                 4.339                  0.004612

In [28]:
# Predict results for 'X_new':
unvpl = lm.predict(X_new)

In [29]:
# Add 'Prediction' column to X_new df with results from previous cell:
X_new['Prediction'] = unvpl.tolist()

#  Add the "Kepler Name (OI)" column to the "planet_predict_df" from "names_df": 
X_new["Kepler_OI_Name"] = names_df

# Rename df:
planet_prediction_df = X_new
print(planet_prediction_df.shape)
planet_prediction_df.head(20)

(1951, 11)


Number_of_Planets  Planet_Radius_Earth  Equalibrium_Temp_K  \
3                    1                14.60               638.0   
38                   1                12.21              1103.0   
59                   1                 7.51               467.0   
63                   2                19.45               734.0   
64                   3                 0.55              1272.0   
85                   1                 7.73               812.0   
93                   1                13.60               643.0   
113                  1                 7.61               511.0   
119                  1                30.09              1155.0   
124                  1                72.77              1947.0   
167                  1                10.59               188.0   
173                  1                14.21               552.0   
177                  1                19.02               733.0   
185                  1                 8.98               366.0   
196                  1                22.94               723.0   
217                  1                12.04               929.0   
231                  1                 4.87               432.0   
232                  1                 4.78               226.0   
254                  1                 1.87              1010.0   
270                  1                 2.42              1194.0   

     Orbit_Period_Days  Star_Radius_Sun  Star_Temp_K  Star_Mass_Sun  \
3            19.899140            0.868       5853.0          0.961   
38            4.959319            1.082       5712.0          0.976   
59           40.419504            0.781       5446.0          0.714   
63            7.240661            0.765       5005.0          0.850   
64            3.435916            1.087       5779.0          0.941   
85           10.181584            0.836       5988.0          0.885   
93           19.620347            0.905       5710.0          0.928   
113          34.843986            0.871       5509.0          0.840   
119           4.640900            0.826       6463.0          0.786   
124           1.028437            0.990       6228.0          0.959   
167         235.998640            0.657       4245.0          0.666   
173          39.748931            0.964       6122.0          1.041   
177          13.610148            0.903       5697.0          0.857   
185         119.020704            1.062       5453.0          0.904   
196          12.940669            0.809       5875.0          0.895   
217           2.688899            0.639       4820.0          0.699   
231          12.732624            0.541       3950.0          0.563   
232          83.924940            0.509       3960.0          0.526   
254           3.886655            0.738       5662.0          0.813   
270           2.494170            0.829       5569.0          0.928   

     Star_Surface_Gravity  Planet_Star_Radius_Ratio  Prediction Kepler_OI_Name  
3                   4.544                  0.154046           0      K00753.01  
38                  4.359                  0.103379           0      K00760.01  
59                  4.507                  0.088069           1      K00777.01  
63                  4.595                  0.232818           0      K00780.02  
64                  4.339                  0.004612           1      K00115.03  
85                  4.541                  0.084708           0      K00797.01  
93                  4.492                  0.137595           0      K00802.01  
113                 4.482                  0.080046           1      K00815.01  
119                 4.500                  0.333559           0      K00820.01  
124                 4.428                  0.673134           0      K00823.01  
167                 4.627                  0.147560           0      K00868.01  
173                 4.487                  0.134949           0      K00856.01  
177                 4.460                  0.192842 

In [30]:
# Count values in 'Prediction' column:
planet_prediction_df['Prediction'].value_counts()

1    1042
0     909
Name: Prediction, dtype: int64

In [31]:
planet_prediction_df['Number_of_Planets'].value_counts()

1    1431
2     351
3     101
4      52
5      12
7       3
6       1
Name: Number_of_Planets, dtype: int64

In [32]:
# Export the Dataframe as a new CSV file without the index.
planet_prediction_df.to_csv("../resources/data/model-output-data/planet_prediction_df.csv", index=False)
planet_prediction_df.dtypes

Number_of_Planets             int64
Planet_Radius_Earth         float64
Equalibrium_Temp_K          float64
Orbit_Period_Days           float64
Star_Radius_Sun             float64
Star_Temp_K                 float64
Star_Mass_Sun               float64
Star_Surface_Gravity        float64
Planet_Star_Radius_Ratio    float64
Prediction                    int64
Kepler_OI_Name               object
dtype: object